In [3]:
import os
import datetime
from pathlib import Path
import glob

folder = "TIRF_IRM"
transform_matrix = "transform_matrix.json"

timestamps = []
tiff_filelist = glob.glob(f"{folder}/*.tiff")
for file in tiff_filelist:
    time = datetime.datetime.fromtimestamp(os.path.getmtime(file)).replace(
        second=0, microsecond=0
    )
    if time not in timestamps:
        timestamps.append(time)

for timestamp in timestamps:
    for file in tiff_filelist:
        time = datetime.datetime.fromtimestamp(os.path.getmtime(file)).replace(
            second=0, microsecond=0
        )
        if time == timestamp:
            if "WT" in file:
                wt_file = file
            else:
                irm_file = file

    output_path = os.path.join(folder, Path(wt_file).stem + "_output")

    !python image_aligner.py {wt_file} {irm_file} -m {transform_matrix} -o {output_path}